# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
vacation_data = pd.read_csv("WeatherPyData.csv")
vacation_data.head()

,lat,long,temp,cloudiness,humidity,wind_speed,city
0,28.37,129.48,25.00,broken clouds,83,7.70,naze
1,35.60,103.21,0.39,few clouds,85,0.31,linxia
2,18.80,110.40,23.97,overcast clouds,90,5.92,wanning
3,68.05,166.44,-6.48,overcast clouds,94,2.00,bilibino
4,68.53,146.42,-18.87,overcast clouds,91,4.44,belaya gora


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# configure gmap
gmaps.configure(api_key=g_key)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_data[["lat", "long"]].astype(float)
humidity = vacation_data["humidity"].astype(int)

In [12]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
vacation_location = vacation_data.loc[(vacation_data['temp'] > 20) & (vacation_data['cloudiness'].isin(['light rain', 'thunderstorm'])),["lat", "long"]].astype(float)

In [18]:
vacation_location

,lat,long
27,-22.97,-42.03
41,21.46,-71.14
52,-13.34,-44.64
54,-21.21,-159.78
77,2.04,45.34
90,-8.77,160.70
109,-22.43,-151.33
118,8.89,-64.25
137,0.52,25.20
169,-2.57,150.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = vacation_location
hotel_df["Hotel Name"] = ""

In [24]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['lat']}, {row['long']}",
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
    "keyword": "hotel"
}   
    
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        row['hotel name']=results[0]['name']
        
        hotel_df.loc[index, 'name'] = results[0]['name']
        hotel_df.loc[index, 'city'] = results[0]['vicinity']
        hotel_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


KeyboardInterrupt: 

In [27]:
results[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 7.770851999999999, 'lng': -72.2208157},
  'viewport': {'northeast': {'lat': 7.772247279892721,
    'lng': -72.21946157010728},
   'southwest': {'lat': 7.769547620107277, 'lng': -72.22216122989272}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Posada Los Nonos',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 810,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101043937795028288484">A Google User</a>'],
   'photo_reference': 'CmRaAAAAOv3rwBX9K-TlhQYEbhGhL8a0GbcFQuQiuWc8O9sUo5NNw7qIxI6sYppfWCop7RPFotxL__1d9im4_0yMVjJU2sO5kOvI4OuSxVSFsCrGJbQ_c8lUbVQTbqsgO-WklzBHEhAIrz0VUVvblVZJ_fLgv14qGhQQ3bEMHWhBwXug5Eyej4yRevXzwg',
   'width': 1080}],
 'place_id': 'ChIJwS69hb1sZo4RBc_hkXbJeeI',
 'plus_code': {'compound_code': 'QQCH+8M San Cristobal, Táchira, Venezuela',
  'global_code': '67V9QQCH+8M'},
 'rating': 3.8,
 'reference': 'ChIJwS69hb1s

In [25]:
hotel_df

,lat,long,Hotel Name
27,-22.97,-42.03,
41,21.46,-71.14,
52,-13.34,-44.64,
54,-21.21,-159.78,
77,2.04,45.34,
90,-8.77,160.70,
109,-22.43,-151.33,
118,8.89,-64.25,
137,0.52,25.20,
169,-2.57,150.80,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
